In [0]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from google.colab import drive
import pickle
from nltk.tokenize import sent_tokenize, word_tokenize

drive.mount('/content/gdrive')


In [0]:
options = { 'node_color': 'black','node_size': 100,'width': 3}

In [0]:
#Translate sentence to a Scene
class Scene():
  #Each sentece is a temporal evolution of graphs
  def __init__(self):
    self.node_list = []
    self.sceneGraph = nx.DiGraph()
    self.temporalListofGraphs = []
    self.timeAlive = 0 #No concept of time (yet)
  
  def addNode(self, node):
    print("Debug: added node: ", node)
    self.node_list.append(node)
    self.sceneGraph.add_node(node)
    
  def drawGraphatTime(self, time):
    print(self.temporalListofGraphs)
    pos = nx.shell_layout(self.temporalListofGraphs[time]);
    nx.draw_random(self.temporalListofGraphs[time], with_labels=True)
    
  def captureMoment(self):
    #NOTE: Only incriment time here and no where else 
    self.temporalListofGraphs.append(self.sceneGraph.copy())
    self.timeAlive = self.timeAlive + 1
    

In [0]:
class Concept():
  #Each concept has a name and a series of ActionMemories
  def __init__(self, cName, a):
    self.conceptName = cName
    self.word_list = []
    self.action = a
    
  def addWordToConcept(self, w):
    self.word_list.append(w)


In [0]:
class Mind():
  #Mind is the object that is saved and restored so contain state-less
  #information here
  def __init__(self):
    self.listOfConcepts = []
  
  def addConcept(self, c):
    self.listOfConcepts.append(c)

class Jasmine():
  def __init__(self):
    #Recover memory
    self.mind = Mind()
    self.retriveMemories()
    self.mainScene = Scene()
    
  def resetMemory(self):
    self.mind = Mind()
    
  def saveMemories(self):
    with open('/content/gdrive/My Drive/memoryState.obj', 'wb') as file:
      pickle.dump(self.mind, file)
      
  def retriveMemories(self):
    with open('/content/gdrive/My Drive/memoryState.obj', 'rb') as file:
      self.mind = pickle.load(file)
      
  def readUserInput(self):
    currentInput = input("You: ")
    #Check if question or scene (./?)
    print("User input ends with: ", currentInput.endswith('.'))
    if currentInput.endswith('.'):
      self.parseSentence2Scene(currentInput)
  
  def sayThis(self, str):
    print("Jasmine: ", str)
    
  def askThis(self, str):
    self.sayThis(str)
    return input("You: ")
    
  def executeThis(self, concept, instanceName):
    if(concept.action == "add node"):
      self.mainScene.addNode(instanceName)
    self.mainScene.captureMoment()
    
  def parseSentence2Scene(self,currentInput):
    found = 0
    words = [word for word in word_tokenize(currentInput)]
    print(words)
    for word in words:
      #Search memory using your mind 
      #if found then execute otherwise learn with human
      print("Looking for: ", word)
      for concept in self.mind.listOfConcepts:
        for wordSeen in concept.word_list:
          if wordSeen == word:
            #Word found in action memory 
            #execute action in the mind
            found = 1
            self.executeThis(concept, word)
      if(found == 0):
        #Word not found ask human L0 question 
        self.sayThis("I don't know what " + word + " means.")
        ans = self.askThis("Can you teach me?")
        #Add to static memory systematically 
        if ans == "Yes" or ans == "yes":
          self.learnConceptWithHuman(word)
  
  def learnConceptWithHuman(self, word):
    ans = self.askThis("Do I Know the Concept?")
    if ans == "Yes" or ans == "yes":
      #Search in concept memory and append word instance
      cName = self.askThis("What is the name of the concept?")
      for concept in self.mind.listOfConcepts:
        if concept.conceptName == cName:
          concept.addWordToConcept(word)
        self.executeThis(concept, word)
    else:
      #create a new concept 
      cName = self.askThis("What is the name of the concept?")
      act = self.askThis("What is the action I am supposed to take?")
      con = Concept(cName, act)
      con.addWordToConcept(word)
      self.mind.addConcept(con)
      self.executeThis(con, word)
      
   
    
  def makeSelfAware(self):
    self.mainScene.add_node("Jasmine")
    self.mainScene.captureMoment()
    
    
    
    

In [0]:
j = Jasmine()
j.readUserInput()
j.saveMemories()


In [0]:
print("Nodes: ", j.mainScene.sceneGraph.nodes)
j.mainScene.drawGraphatTime(1)

In [0]:
import nltk
nltk.download()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------

Download which package (l=list; x=cancel)?
       | 
       | Downloading package abc to /root/nltk_data...
       |   Unzipping corpora/abc.zip.
       | Downloading package alpino to /root/nltk_data...
       |   Unzipping corpora/alpino.zip.
       | Downloading package biocreative_ppi to /root/nltk_data...
       |   Unzipping corpora/biocreative_ppi.zip.
       | Downloading package brown to /root/nltk_data...
       |   Unzipping corpora/brown.zip.
       | Downloading package brown_tei to /root/nltk_data...
       |   Unzipping corpora/brown_tei.zip.
       | Downloading package cess_cat to /root/nltk_data...
       |   Unzipping corpora/cess_cat.zip.
       | Downloading package cess_esp to /root/nltk_data...
       |   Unzipping corpora/cess